In [9]:
from PIL import Image
import os
from tqdm import tqdm

In [10]:
def create_image_faster(name_of_file,b_string,image_index,image=None,point=None):
    if(image==None):
        image = Image.new('RGB', (640, 480), (256, 256, 256))
        width, height = image.size
        point={'x':0,'y':0}
    else:
        width, height = image.size
    all_pixels={'Red':0,'Green':0,'Blue':0}
    if(point['x']+1<width and point['y']+1<height):
        for i in range(3):
            binary_substring=b_string[:8]
            b_string=b_string[8:]
            decimal_temp = int(binary_substring, 2)
            all_pixels[list(all_pixels.keys())[i]]=decimal_temp
        image.putpixel((point['x'], point['y']), (all_pixels['Red'], all_pixels['Green'], all_pixels['Blue']))

        point['x']+=1
    elif(point['x']+1==width and point['y']+1<height):
        for i in range(3):
            binary_substring=b_string[:8]
            b_string=b_string[8:]
            decimal_temp = int(binary_substring, 2)
            all_pixels[list(all_pixels.keys())[i]]=decimal_temp
        image.putpixel((point['x'], point['y']), (all_pixels['Red'], all_pixels['Green'], all_pixels['Blue']))

        point['x']=0
        point['y']+=1
    elif(point['x']+1==width and point['y']+1==height):
        image.save('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(image_index)+'.jpg')
        image=None
        point=None
        image_index+=1
    return image,point,image_index

In [11]:
def convert_file_to_binary(name_of_file,path):
    
    
    with open(path, 'rb') as f:
        binary_data = f.read()
        # binary_string = ''.join(format(byte, '08b') for byte in binary_data)
        binary_string=""
        previous_string=None
        current_index=0;
        image=None
        img_index=0;
        point=None
        #add tqdm to this loop
        for i in tqdm(range(len(binary_data)),desc="Converting to binary"):
        # for i in range(len(binary_data)):

            temp= format(binary_data[i], '08b')
            binary_string+=temp
            if((i+1)%3==0):
                # print(f'image_index={img_index}')
                image,point,img_index=create_image_faster(name_of_file,binary_string,img_index,image,point)
                if(image==None):
                    print(f'image_index --end-- ={img_index}')
                    image,point,img_index=create_image_faster(name_of_file,binary_string,img_index,image,point)
    



In [12]:
def cal_no_of_images(name_of_file):
    directory = './images/'
    file_list = os.listdir(directory)
    num_files=0
    for file in file_list:
        if name_of_file == str(file)[:-16]:
            print("images are : "+str(file))
            num_files+=1

    return num_files

In [13]:
def CreateVideo(name_of_file):
    print("Creating video...")
    import imageio.v2
    writer = imageio.v2.get_writer('./videos/'+name_of_file+'.mp4', fps=30)
    noi=cal_no_of_images(name_of_file)
    for i in range(noi):
        print("Video: images are : "+str(i))
        image = imageio.v2.imread('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(i+1)+'.jpg')
        writer.append_data(image)

    writer.close()

In [14]:
def Delete_Images(name_of_file):
    import os
    directory = './images/'
    file_list = os.listdir(directory)
    for file in file_list:
        if name_of_file == str(file)[:-16]:
            os.remove('./images/'+file)
            print("Deleted : "+str(file))

In [15]:
def convertdatatovideo(name_of_file,path):
    convert_file_to_binary(name_of_file,path)
    CreateVideo(name_of_file)
    Delete_Images(name_of_file)

In [16]:
directory = './inputs/'
file_list = os.listdir(directory)
for file in file_list:
    convertdatatovideo(str(file),directory+str(file))

Creating video...
